In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import cv2

# Load the images and labels
images = []
labels = []

with open("dataset.txt", "r") as file:
    for line in file:
        line = line.strip().split()
        images.append(cv2.imread(line[0]))
        labels.append(line[1:])

# Convert the images and labels to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Affine(rotate=(-45, 45)),  # rotate the images by a random degree between -45 and 45
    iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),  # add random noise to the images
    iaa.Multiply((0.8, 1.2)),  # adjust the brightness of the images
])

# Augment the images and labels
images_aug, labels_aug = seq(images=images, annotations=labels)

# Save the augmented images and labels
for i, (image, label) in enumerate(zip(images_aug, labels_aug)):
    cv2.imwrite("augmented_images/image_{}.jpg".format(i), image)
    with open("augmented_labels/label_{}.txt".format(i), "w") as file:
        file.write(" ".join(label))

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Load the dataset
df = pd.read_csv("dataset.csv")

# Separate the majority and minority classes
df_majority = df[df.target.isin([0, 1])]
df_minority = df[df.target.isin([2, 3, 4])]

# Oversample the minority class
df_minority_oversampled = resample(df_minority,
                                    replace=True,
                                    n_samples=len(df_majority),
                                    random_state=123)

# Undersample the majority class
df_majority_undersampled = resample(df_majority,
                                     replace=False,
                                     n_samples=len(df_minority),
                                     random_state=123)

# Combine the oversampled minority class with the undersampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority_oversampled])

# Shuffle the rows
df_balanced = df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)

# Print the balanced dataset
print(df_balanced)
